One can wonder the following question: what is the probability $p_i$ of an elevator being at the ith floor when it is called at a time $t$ of the day?

To model this problem let us consider dividing the day into 24 blocks, each representing one hour. For example, the block $b_1$ corresponds to times beteween 00:00 and 01:00, $b_2$ between 01:00 and 02:00, and so on. Thus, $p_i(b_j)$ is the probaility for the elevator to be avilable when it is called at a given time in the block $b_j$. For a $N$-floors building, we aim to determine $p_1(b_j), \ldots, p_N(b_j)$. Then, the ideal resting floor at a given block of time $b_j$ is the one for which the probabilty is maximum.  

In practice, one can record the time $t$ of a call and the current resting floor if the elevator is available. It might happen that the the elevator is moving when the a call occurs. In that case, the resting floor is the one that becomes available for that call. Then, we can train a model with this data and once it is trained we expect to predict $p_1(b_j), \ldots, p_N(b_j)$ for a given input $b_j$.

For a 20 floors building, I simulated some data using the codes in `data_generation.py` for an entire year. I added weights to the calling floors in `pcalling` and resting floors in `presting`. The code loops day by day over the entire year, generating `resting_floor` and `calling_floor` according to weights. Some floors are biased un purpose, to simulate a realistic situation. This data is stored in the table `elevator` within `database.db` (a sqlite datase).

In this notebook I do some queries on the data that might result relevant for feeding a prediction engine.

In [105]:
import sqlite3
import datetime
import random
import numpy as np

con = sqlite3.connect("database.db")
cur = con.cursor()

### Let us read some data

In [106]:
# Get information about the table "elevator"
result = cur.execute("PRAGMA table_info(elevator)")
result.fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'calling_floor', 'INTEGER', 0, None, 0),
 (2, 'resting_floor', 'INTEGER', 0, None, 0),
 (3, 'time', 'TIMESTAMP', 0, None, 0),
 (4, 'time_block', 'INTEGER', 0, None, 0)]

In [107]:
# Get the number of rows in the table "elevator"
total_rows = cur.execute("SELECT COUNT(*) FROM elevator;").fetchall()[0][0]
print(f"There are {total_rows} entries in \"elevator\"")

There are 84680 entries in "elevator"


In [108]:
# Let's see how many times a demand was made from a given floor
floor = 6
result = cur.execute("""SELECT COUNT(*) FROM elevator WHERE calling_floor = ?""",(floor,)).fetchall()[0][0]
print(f"{result} calls were made from the {floor}th floor")

43586 calls were made from the 6th floor


In [109]:
# How many times a given floor was the resting floor
floor = 15
result = cur.execute("""SELECT COUNT(*) FROM elevator WHERE resting_floor = ?""",(floor,)).fetchall()[0][0]
print(f"{result} times {floor} was the resting floor")

45456 times 15 was the resting floor


Let us try seeing how many times a demand was made from a floor while the elevator was resting at some other floor during a range of time blocks

In [110]:
l = 2
u = 15
calling_floor = 17
resting_floor = 10 
values = (calling_floor, resting_floor, l,u)

query = """
        SELECT COUNT(*) 
        FROM elevator 
        WHERE calling_floor = ? AND resting_floor = ? AND (time_block BETWEEN ?  AND ?)
        ;
        """

result = cur.execute(query, values).fetchall()[0][0]
print(f"Total of {result} entries with those constraints")

Total of 6 entries with those constraints


Let us put this data into numpy arrays, a suitable format to feed and train a prediction engine. Let us suppouse that we onw to use the `time blocks` and `calling floors` as inputs and the `resting floors` as outputs.

In [111]:
input_data = np.zeros((2,total_rows))
output_data = np.zeros(total_rows)

for i in range(1,total_rows+1):
    _,cfloor,rfloor,_,block = cur.execute("SELECT * FROM elevator WHERE id = ?", (i,)).fetchall()[0]
    input_data[:,i-1] = np.array([cfloor,block])
    output_data[i-1] = rfloor


In [112]:
input_data

array([[ 6.,  6.,  6., ...,  6.,  6., 18.],
       [ 0.,  0.,  0., ..., 23., 23., 23.]])

In [113]:
output_data

array([18., 10., 15., ...,  1., 15., 13.])